In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import datetime
from sklearn.metrics import accuracy_score,f1_score
%load_ext tensorboard

In [ ]:
!rm -rf ./logs/

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
train_df=pd.read_csv('gdrive/My Drive/NLP_Project/train_data_SMOTE.csv')
train_df=train_df.drop(columns=train_df.columns[0])

test_df=pd.read_csv('gdrive/My Drive/NLP_Project/test_data_SMOTE.csv')
test_df=test_df.drop(columns=test_df.columns[0])

Mounted at /content/gdrive


In [ ]:
X_train=train_df.iloc[:,:-1]
y_train=train_df.iloc[:,-1]

X_test=test_df.iloc[:,:-1]
y_test=test_df.iloc[:,-1]

In [ ]:
X_train=X_train.to_numpy()
y_train=y_train.to_numpy()

X_test=X_test.to_numpy()
y_test=y_test.to_numpy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder


onehot_encoder = OneHotEncoder(categories=[np.arange(40)],sparse_output=False)
y_train_onehot=onehot_encoder.fit_transform(np.array(y_train).reshape(-1,1))
y_test_onehot=onehot_encoder.transform(np.array(y_test).reshape(-1,1))

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.F1Score(name='F1_score',average='micro')
]

In [ ]:

model_ANN=tf.keras.Sequential([
    tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(40,activation='softmax'),
],name="ANN_model")

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.F1Score(name='F1_score',average='micro')
]

model_ANN.compile(optimizer='adam',loss='categorical_crossentropy',metrics=METRICS)
log_dir = "logs/fit/" + model_ANN.name+"-"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_ANN.fit(X_train,y_train_onehot,epochs=3,batch_size=128,validation_data=(X_test,y_test_onehot),callbacks=[tensorboard_callback])


Epoch 1/3
38/38 [==============================] - 3s 17ms/step - loss: 3.1324 - accuracy: 0.9765 - F1_score: 0.2704 - val_loss: 3.1164 - val_accuracy: 0.9747 - val_F1_score: 0.2050
Epoch 2/3
38/38 [==============================] - 0s 7ms/step - loss: 2.2589 - accuracy: 0.9794 - F1_score: 0.4116 - val_loss: 2.7147 - val_accuracy: 0.9742 - val_F1_score: 0.3010
Epoch 3/3
38/38 [==============================] - 0s 6ms/step - loss: 1.9104 - accuracy: 0.9795 - F1_score: 0.4785 - val_loss: 2.4459 - val_accuracy: 0.9733 - val_F1_score: 0.3340


In [ ]:
model_ANN_dropout=tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),  # Increase dropout rate
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),  # Increase dropout rate
    tf.keras.layers.Dense(40, activation='softmax')
])
model_ANN_dropout.compile(optimizer='adam',loss='categorical_crossentropy',metrics=METRICS)
model_ANN_dropout.fit(X_train,y_train_onehot,epochs=10,batch_size=16,validation_data=(X_test,y_test_onehot))

In [ ]:
model_ANN_improved = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(40, activation='softmax')
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model_ANN_improved.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=METRICS)

model_ANN_improved.fit(X_train,y_train_onehot,epochs=5,batch_size=32,validation_data=(X_test,y_test_onehot),callbacks=[tensorboard_callback])

Epoch 1/5
152/152 [==============================] - 4s 9ms/step - loss: 8.7632 - accuracy: 0.9761 - F1_score: 0.2396 - val_loss: 7.9005 - val_accuracy: 0.9750 - val_F1_score: 0.2060
Epoch 2/5
152/152 [==============================] - 1s 7ms/step - loss: 6.5511 - accuracy: 0.9785 - F1_score: 0.3995 - val_loss: 6.4431 - val_accuracy: 0.9750 - val_F1_score: 0.2590
Epoch 3/5
152/152 [==============================] - 1s 7ms/step - loss: 5.2032 - accuracy: 0.9788 - F1_score: 0.4410 - val_loss: 5.2043 - val_accuracy: 0.9750 - val_F1_score: 0.3120
Epoch 4/5
152/152 [==============================] - 1s 7ms/step - loss: 4.2293 - accuracy: 0.9788 - F1_score: 0.4606 - val_loss: 4.2593 - val_accuracy: 0.9745 - val_F1_score: 0.3360
Epoch 5/5
152/152 [==============================] - 1s 7ms/step - loss: 3.5359 - accuracy: 0.9788 - F1_score: 0.4709 - val_loss: 3.5936 - val_accuracy: 0.9739 - val_F1_score: 0.3470


CNN

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)

In [ ]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
model_CNN = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=15, kernel_size=8, input_shape=(473, 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2, padding='valid'),
    tf.keras.layers.Dropout(0.5),  # Add dropout after pooling

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(40, activation="softmax")
], name="CNN_model")

model_CNN.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.experimental.Adamax(), metrics=METRICS)

# Implementing early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model_CNN.fit(X_train, y_train_onehot, epochs=20, validation_data=(X_test, y_test_onehot), callbacks=[early_stopping])


Epoch 1/20
152/152 [==============================] - 2s 5ms/step - loss: 3.2623 - accuracy: 0.9750 - F1_score: 0.2911 - val_loss: 3.2740 - val_accuracy: 0.9750 - val_F1_score: 0.2040
Epoch 2/20
152/152 [==============================] - 1s 4ms/step - loss: 2.7457 - accuracy: 0.9760 - F1_score: 0.3549 - val_loss: 3.2242 - val_accuracy: 0.9749 - val_F1_score: 0.2060
Epoch 3/20
152/152 [==============================] - 1s 4ms/step - loss: 2.6069 - accuracy: 0.9783 - F1_score: 0.3607 - val_loss: 3.1428 - val_accuracy: 0.9750 - val_F1_score: 0.2070
Epoch 4/20
152/152 [==============================] - 1s 4ms/step - loss: 2.5181 - accuracy: 0.9789 - F1_score: 0.3687 - val_loss: 3.0775 - val_accuracy: 0.9749 - val_F1_score: 0.2080
Epoch 5/20
152/152 [==============================] - 1s 4ms/step - loss: 2.4512 - accuracy: 0.9791 - F1_score: 0.3741 - val_loss: 3.0654 - val_accuracy: 0.9748 - val_F1_score: 0.2070
Epoch 6/20
152/152 [==============================] - 1s 4ms/step - loss: 2.3948

In [ ]:
model_Deep_LSTM = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(473, 1), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(40, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(0.01))
], name="Improved_Deep_LSTM_model")

model_Deep_LSTM.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=METRICS)

model_Deep_LSTM.fit(X_train, y_train_onehot, epochs=20, batch_size=64, validation_data=(X_test, y_test_onehot),)





Epoch 1/20
76/76 [==============================] - 11s 59ms/step - loss: 4.0101 - accuracy: 0.9750 - F1_score: 0.0743 - val_loss: 3.7591 - val_accuracy: 0.9750 - val_F1_score: 0.0030
Epoch 2/20
76/76 [==============================] - 3s 44ms/step - loss: 3.7498 - accuracy: 0.9750 - F1_score: 0.0820 - val_loss: 3.6338 - val_accuracy: 0.9750 - val_F1_score: 0.0030
Epoch 3/20
76/76 [==============================] - 3s 43ms/step - loss: 3.5229 - accuracy: 0.9750 - F1_score: 0.1536 - val_loss: 3.5286 - val_accuracy: 0.9750 - val_F1_score: 0.0030
Epoch 4/20
76/76 [==============================] - 3s 44ms/step - loss: 3.3193 - accuracy: 0.9750 - F1_score: 0.2027 - val_loss: 3.4232 - val_accuracy: 0.9750 - val_F1_score: 0.0450
Epoch 5/20
76/76 [==============================] - 3s 43ms/step - loss: 3.1889 - accuracy: 0.9750 - F1_score: 0.2124 - val_loss: 3.3167 - val_accuracy: 0.9750 - val_F1_score: 0.2060
Epoch 6/20
76/76 [==============================] - 3s 43ms/step - loss: 3.1537 - ac

In [ ]:
model_BiLSTM = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True), input_shape=(473, 1)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.3),



    tf.keras.layers.Dense(40, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(0.01))
], name="BiLSTM_model")

model_BiLSTM.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=METRICS)

model_BiLSTM.fit(X_train, y_train_onehot, epochs=20, batch_size=32, validation_data=(X_test, y_test_onehot))


Epoch 1/20
152/152 [==============================] - 16s 69ms/step - loss: 3.2976 - accuracy: 0.9747 - F1_score: 0.2337 - val_loss: 3.3469 - val_accuracy: 0.9746 - val_F1_score: 0.1590
Epoch 2/20
152/152 [==============================] - 9s 59ms/step - loss: 2.7644 - accuracy: 0.9748 - F1_score: 0.3371 - val_loss: 3.0013 - val_accuracy: 0.9750 - val_F1_score: 0.2220
Epoch 3/20
152/152 [==============================] - 9s 60ms/step - loss: 2.5707 - accuracy: 0.9755 - F1_score: 0.3759 - val_loss: 2.9325 - val_accuracy: 0.9730 - val_F1_score: 0.2890
Epoch 4/20
152/152 [==============================] - 9s 60ms/step - loss: 2.4207 - accuracy: 0.9768 - F1_score: 0.4139 - val_loss: 2.8044 - val_accuracy: 0.9733 - val_F1_score: 0.2990
Epoch 5/20
152/152 [==============================] - 9s 59ms/step - loss: 2.3376 - accuracy: 0.9770 - F1_score: 0.4321 - val_loss: 2.8388 - val_accuracy: 0.9725 - val_F1_score: 0.2670
Epoch 6/20
152/152 [==============================] - 9s 60ms/step - loss:

In [ ]:
import tensorflow as tf


class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.units = units

        # Initialize weight matrices
        self.W_q = tf.keras.layers.Dense(units)
        self.W_k = tf.keras.layers.Dense(units)
        self.W_v = tf.keras.layers.Dense(units)

        self.scale = tf.keras.layers.Lambda(lambda x: x / tf.math.sqrt(tf.cast(units, tf.float32)))

    def call(self, inputs):
        query, key, value = inputs

        # Process inputs through weight matrices
        query = self.W_q(query)
        key = self.W_k(key)
        value = self.W_v(value)

        # Calculate attention scores
        score = tf.matmul(query, key, transpose_b=True)
        score = self.scale(score)
        attention_weights = tf.nn.softmax(score, axis=-1)

        # Apply attention weights to value
        output = tf.matmul(attention_weights, value)

        return output


# Define the BiLSTM model with Attention
input_layer = tf.keras.Input(shape=(473, 1))
lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(input_layer)
lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(lstm_1)

attention_output = Attention(32)([lstm_2, lstm_2, lstm_2])  # Pass the same input to all attention components

concatenate=tf.keras.layers.Concatenate(axis=1)([attention_output])

output_layer = tf.keras.layers.Dense(40, activation="softmax")(concatenate)



model = tf.keras.Model(inputs=input_layer, outputs=output_layer)


# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=METRICS)
# model.fit(X_train, y_train_onehot, epochs=20, batch_size=32, validation_data=(X_test, y_test_onehot))


In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=METRICS)

In [ ]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 473, 1)]             0         []                            
                                                                                                  
 bidirectional_52 (Bidirect  (None, 473, 256)             133120    ['input_16[0][0]']            
 ional)                                                                                           
                                                                                                  
 bidirectional_53 (Bidirect  (None, 473, 128)             164352    ['bidirectional_52[0][0]']    
 ional)                                                                                           
                                                                                            

In [ ]:
model.fit(X_train,y_train_onehot, epochs=20, batch_size=32, validation_data=(X_test, y_test_onehot))

Epoch 1/20
152/152 [==============================] - 25s 101ms/step - loss: 3.0022 - accuracy: 0.9750 - F1_score: 0.1453 - val_loss: 3.2844 - val_accuracy: 0.9750 - val_F1_score: 0.2060
Epoch 2/20
152/152 [==============================] - 13s 88ms/step - loss: 2.8803 - accuracy: 0.9750 - F1_score: 0.1879 - val_loss: 3.2556 - val_accuracy: 0.9750 - val_F1_score: 0.2060
Epoch 3/20
 58/152 [==========>...................] - ETA: 7s - loss: 2.8164 - accuracy: 0.9750 - F1_score: 0.2398

KeyboardInterrupt: ignored

Machine Learning models

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

svm_classifier = SVC(kernel='linear', decision_function_shape='ovr')  # Using a linear kernel for example
svm_classifier.fit(X_train, y_train)

# Predict on the test set
predictions = svm_classifier.predict(X_test)

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, predictions)
f1_score_micro = f1_score(y_test, predictions, average='micro')
print(f"Accuracy: {accuracy * 100:.2f}%, F1 Score (micro): {f1_score_micro * 100:.2f}%")

Accuracy: 31.60%, F1 Score (micro): 31.60%
